In [17]:
"""
1. 이미지를 resnet에 넣고 npz 파일로 저장
datastructure
{
  'front_view': np.ndarray of shape [total_length, T, 512],
  'top_view':   np.ndarray of shape [total_length, T, 512],
  'wrist_view': np.ndarray of shape [total_length, T, 512],
}

"""
import os
import re
import torch
import numpy as np
import pickle
from PIL import Image
from tqdm import tqdm
from torchvision import models, transforms

device = "cuda" if torch.cuda.is_available() else "cpu"

# ResNet18 feature extractor
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = torch.nn.Identity()  # remove final fc
resnet18 = resnet18.to(device)
resnet18.eval()
#print("Weight hash:", hash(str(resnet18.state_dict())))
# Image transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225]),
])

def extract_features(image_dir):
    def extract_index(filename):
        match = re.search(r'front_view_(\d+)\.png', filename)
        return int(match.group(1)) if match else -1

    image_files = sorted(
        [f for f in os.listdir(image_dir) if f.endswith(".png")],
        key=extract_index
    )

    features = []

    for fname in image_files:
        #print(fname)
        img = Image.open(os.path.join(image_dir, fname)).convert('RGB')
        img_tensor = transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            feat = resnet18(img_tensor)
        features.append(feat.cpu().numpy()[0])

    return np.stack(features, axis=0)  # shape: [T, 512]

def get_episode_number(ep_dir_name):
    match = re.search(r"success_episode(\d+)_steps(\d+)", ep_dir_name)
    return int(match.group(1)) if match else -1

def process_all_episodes(root_dir, output_npz):
    front_list, top_list, wrist_list = [], [], []

    episode_dirs = sorted(
        [d for d in os.listdir(root_dir) if d.startswith("success_episode")],
        key=get_episode_number
    )
   
    for i, ep_dir in enumerate(tqdm(episode_dirs, desc="Processing episodes")):
        ep_path = os.path.join(root_dir, ep_dir)
        print(f"[{i+1}/{len(episode_dirs)}] Processing: {ep_dir}")

        try:
            front_feat = extract_features(os.path.join(ep_path, "front_view"))
            top_feat   = extract_features(os.path.join(ep_path, "top_view"))
            wrist_feat = extract_features(os.path.join(ep_path, "wrist_view"))
        except Exception as e:
            print(f"❌ Error in {ep_dir}: {e}")
            continue

        # Optionally check consistency of T
        T_f, T_t, T_w = len(front_feat), len(top_feat), len(wrist_feat)
        if T_f != T_t or T_t != T_w:
            print(f"⚠️ Skipping {ep_dir} due to mismatched frame counts: front={T_f}, top={T_t}, wrist={T_w}")
            continue

        front_list.append(front_feat)
        top_list.append(top_feat)
        wrist_list.append(wrist_feat)

    # Save as object arrays (list of np.ndarray)
    np.savez_compressed(
        output_npz,
        front_view=np.array(front_list, dtype=object),
        top_view=np.array(top_list, dtype=object),
        wrist_view=np.array(wrist_list, dtype=object)
    )
    print(f"✅ Saved to {output_npz}")


# 실행
process_all_episodes("/AILAB-summer-school-2025/success_data_raw", "dataset/image_features(resnet18).npz")


Processing episodes:   0%|          | 0/505 [00:00<?, ?it/s]

[1/505] Processing: success_episode1_steps320


Processing episodes:   0%|          | 1/505 [00:00<06:12,  1.35it/s]

[2/505] Processing: success_episode2_steps316


Processing episodes:   0%|          | 2/505 [00:01<06:12,  1.35it/s]

...

[421/505] Processing: success_episode421_steps342


Processing episodes:  83%|████████▎ | 421/505 [03:59<00:45,  1.83it/s]

[422/505] Processing: success_episode422_steps326


Processing episodes:  83%|████████▎ | 421/505 [04:00<00:47,  1.75it/s]


KeyboardInterrupt: 

In [18]:

"""
2. npz 파일 shape확인
"""
import numpy as np

# 파일 경로
file_path = "/AILAB-summer-school-2025/scripts/make_dataset/image_features(resnet18).npz"

data = np.load(file_path, allow_pickle=True)

for key in data.files:
    array = data[key]
    print(f"🔍 Checking key: {key}")

    # 요소가 1차원 object 배열일 경우 (비정형 데이터)
    if array.dtype == object:
        shapes = [np.shape(v) for v in array]
    else:
        shapes = [np.shape(row) for row in array]

    unique_shapes = set(shapes)
    if len(unique_shapes) == 1:
        print(f"✅ All {len(shapes)} entries in '{key}' have the same shape: {unique_shapes.pop()}")
    else:
        print(f"❌ Inconsistent shapes in '{key}':")
        for i, s in enumerate(shapes):
            print(f"  - Index {i}: shape = {s}")

🔍 Checking key: front_view
❌ Inconsistent shapes in 'front_view':
  - Index 0: shape = (64, 512)
  - Index 1: shape = (64, 512)
  - Index 2: shape = (62, 512)
  - Index 3: shape = (63, 512)
  - Index 4: shape = (63, 512)
  - Index 5: shape = (62, 512)
  - Index 6: shape = (66, 512)
  - Index 7: shape = (62, 512)
  - Index 8: shape = (67, 512)
  - Index 9: shape = (64, 512)
  - Index 10: shape = (71, 512)
  - Index 11: shape = (64, 512)
  - Index 12: shape = (72, 512)
  - Index 13: shape = (65, 512)
  - Index 14: shape = (63, 512)
  - Index 15: shape = (65, 512)
  - Index 16: shape = (61, 512)
  - Index 17: shape = (63, 512)
  - Index 18: shape = (62, 512)
  - Index 19: shape = (64, 512)
  - Index 20: shape = (67, 512)
  - Index 21: shape = (66, 512)
  - Index 22: shape = (63, 512)
  - Index 23: shape = (62, 512)
  - Index 24: shape = (68, 512)
  - Index 25: shape = (63, 512)
  - Index 26: shape = (66, 512)
  - Index 27: shape = (67, 512)
  - Index 28: shape = (61, 512)
  - Index 29: sh

In [31]:
import os
import re
import numpy as np
import torch
from PIL import Image
from torchvision import models, transforms

# ✅ ResNet18 정의 (512차원 출력)
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = torch.nn.Identity()
resnet18.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
resnet18 = resnet18.to(device)
print("Weight hash:", hash(str(resnet18.state_dict())))

# ✅ 이미지 transform 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225]),
])

# ✅ 설정
npz_path = "/AILAB-summer-school-2025/scripts/make_dataset/image_features(resnet18).npz"
success_data_raw = "/AILAB-summer-school-2025/success_data_raw"
episode_idx = 25  # 검증할 에피소드 인덱스
stride = 5       # 저장된 프레임 간격
view_names = ["front_view", "top_view", "wrist_view"]

# ✅ .npz 로딩
data = np.load(npz_path, allow_pickle=True)

# ✅ 에피소드 디렉토리 정렬
episode_dirs = sorted(
    [d for d in os.listdir(success_data_raw) if d.startswith("success_episode")],
    key=lambda name: int(re.search(r"success_episode(\d+)", name).group(1))
)
episode_name = episode_dirs[episode_idx]
episode_path = os.path.join(success_data_raw, episode_name)

print(f"🔍 Checking episode: {episode_name}, stride={stride}")

# ✅ View 별 점검 루프
for view in view_names:
    print(f"\n🔎 Checking view: {view}")
    ep_feat_seq = data[view][episode_idx]  # shape [T, 512]
    T = ep_feat_seq.shape[0]
    view_dir = os.path.join(episode_path, view)

    num_fail = 0
    for i in range(T):
        frame_number = i * stride
        filename = f"{view}_{frame_number}.png"
        filepath = os.path.join(view_dir, filename)

        if not os.path.exists(filepath):
            print(f"❌ Missing file: {filepath}")
            num_fail += 1
            continue

        image = Image.open(filepath).convert("RGB")
        img_tensor = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            feat_from_resnet = resnet18(img_tensor).cpu().numpy().squeeze()

        feat_from_npz = ep_feat_seq[i]
        is_same = np.allclose(feat_from_npz, feat_from_resnet, atol=1e-6)

        if not is_same:
            diff = np.abs(feat_from_npz - feat_from_resnet)
            print(f"⚠️  Frame {i} (image {frame_number}.png): mismatch")
            print(f"   → Max diff: {np.max(diff):.6f}, Mean diff: {np.mean(diff):.6f}")
            num_fail += 1

    if num_fail == 0:
        print(f"✅ {view}: All features match.")
    else:
        print(f"❌ {view}: {num_fail}/{T} frames failed consistency check.")


Weight hash: -1720887647942330811
🔍 Checking episode: success_episode26_steps313, stride=5

🔎 Checking view: front_view
✅ front_view: All features match.

🔎 Checking view: top_view
✅ top_view: All features match.

🔎 Checking view: wrist_view
✅ wrist_view: All features match.


In [ ]:
""